In [ ]:
import pandas as pd
import numpy as np
from bertopic import BERTopic
import sqlite3
import sqllite_handler
import copy

In [ ]:
conn = sqlite3.connect('tables/texts_69_76.db')
cur = conn.cursor()

res = cur.execute("SELECT TEXT FROM transcript")
fetched = res.fetchall()
free_text_list = list(map(lambda x: x[0], fetched))

topic_model = BERTopic.load("plots/topic_model_69_76")

doc_df = pd.read_csv('tables/doc_69_76.csv')

In [ ]:
messy_doc_topic_df = topic_model.get_document_info(free_text_list)

topic_desc_df = messy_doc_topic_df[['Name','Top_n_words']].drop_duplicates(ignore_index=True)
doc_topic_df = pd.DataFrame({'id_to_text':doc_df['id_to_text'],'assigned_topic':messy_doc_topic_df['Name']})

topic_desc_df.to_csv('tables/topic_descp_69_76.csv')
doc_topic_df.to_csv('tables/doc_topic_69_76.csv')

### construction below

In [ ]:
import gensim
import pyLDAvis.gensim_models as gensimvis

In [ ]:
from transformers import pipeline

triplet_extractor = pipeline('text2text-generation', model='Babelscape/rebel-large', tokenizer='Babelscape/rebel-large')

In [ ]:
txt ='Luis lives in Zurich.'

out_ = triplet_extractor(txt, return_tensors=True, return_text=False)[0]["generated_token_ids"]

In [ ]:
extracted_text = triplet_extractor.tokenizer.batch_decode(out_['output_ids'][0])

extracted_triplets = extract_triplets(extracted_text[0])
print(extracted_triplets)

In [ ]:
# Function to parse the generated text and extract the triplets
def extract_triplets(text):
    triplets = []
    relation, subject, relation, object_ = '', '', '', ''
    text = text.strip()
    current = 'x'
    for token in text.replace("<s>", "").replace("<pad>", "").replace("</s>", "").split():
        if token == "<triplet>":
            current = 't'
            if relation != '':
                triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
                relation = ''
            subject = ''
        elif token == "<subj>":
            current = 's'
            if relation != '':
                triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
            object_ = ''
        elif token == "<obj>":
            current = 'o'
            relation = ''
        else:
            if current == 't':
                subject += ' ' + token
            elif current == 's':
                object_ += ' ' + token
            elif current == 'o':
                relation += ' ' + token
    if subject != '' and relation != '' and object_ != '':
        triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
    return triplets


In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')

def remove_persons(txt):
    document = nlp(txt)

    edited_txt = ""
    for ent in document:
        
        if ent.ent_type_=='PERSON':
            if ent.whitespace_:
                edited_txt += 'Person'+ ' '
            else:
                edited_txt += 'Person'
        else:
            if ent.whitespace_:
                edited_txt += ent.text+ ' '
            else:
                edited_txt += ent.text
    
    return edited_txt

In [ ]:
import pandas as pd
doc_df = pd.read_csv('tables/doc_69_76v30.csv')

In [ ]:
txt_list = doc_df['text'].values

In [ ]:
processed_txt_list = list(map(lambda x: remove_persons(x), txt_list))

In [ ]:
processed_txt_list[10]

In [ ]:
from keybert import KeyBERT

# Extract keywords
kw_model = KeyBERT()
keywords = kw_model.extract_keywords(txt_list[0])



In [ ]:
import pickle

with open("plots/free_text_list", "rb") as fp:
        free_text_list = pickle.load(fp)

In [ ]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [ ]:
sentences = ["Relations between Greece and Turkey began in the 1830s following Greece's formation after its declaration of independence from the Ottoman Empire. Modern relations began when Turkey declared its formation in 1923 following the defeat of the Ottoman Empire in World War I. \
Greece and Turkey have a rivalry with a history of events that have been used to justify their nationalism.[1][2] These events include the population exchange between Greece and Turkey, the Istanbul pogrom and Cypriot intercommunal violence. Greek-Turkish feuding was not a significant factor in international relations from 1930 to 1955, and during the Cold War, domestic and bipolar politics limited competitive behaviour against each other.[3][4] By the mid-1990s and later decades, the restraint on their rivalry was removed, and both nations had become each other's biggest security risk.[5][6]\
Control of the eastern Mediterranean and Aegean seas remain the basis of the countries' rivalry. Following the end of World War II, the UNCLOS treaty, the decolonisation of Cyprus, and the addition of the Dodecanese to Greece's territory have caused turbulence in the relationship. Several issues frequently affect their current relations, including territorial disputes over the sea and air, minority rights, and Turkey's relationship with the European Union (EU) and its member states—especially Cyprus.[7][8] Control of energy pipelines is also an increasing focus in their relations."]

In [ ]:
for sentence in sentences:
    sid = SentimentIntensityAnalyzer()
    print(sentence)
    ss = sid.polarity_scores(sentence)
    for k in sorted(ss):
        print('{0}: {1}, '.format(k, ss[k]), end='')
    print()

### keyword-ne KG below

In [307]:
import os
import yake
import sklearn.feature_extraction.text as text
import spacy
import pickle 
from sklearn.decomposition import PCA
from sklearn.manifold import Isomap
import numpy as np
import matplotlib.pyplot as plt
from graphdatascience import GraphDataScience
from sklearn.decomposition import PCA
from sklearn.manifold import Isomap, MDS
import numpy as np
import pandas as pd
import math
import copy
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

nlp = spacy.load('en_core_web_sm')

/Users/gokberk/miniconda3/envs/ml4hc_project2/lib/python3.9/site-packages/spacy/util.py:877: UserWarning:

[W095] Model 'en_core_web_sm' (3.2.0) was trained with spaCy v3.2 and may not be 100% compatible with the current version (3.5.0). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate



In [158]:
def apply_ner(sentence):

    doc = nlp(sentence)

    named_entities = []

    entity_chunk = None #(entity,type)

    for token in doc:
        if token.ent_iob_ == 'O':
            if entity_chunk:
                named_entities.append(entity_chunk)
                entity_chunk = None
        elif token.ent_iob_ == 'B':
            if entity_chunk:
                named_entities.append(entity_chunk)
                entity_chunk = None
            entity_chunk = (token.text,token.ent_type_)
        else:
            entity_chunk_text = entity_chunk[0]
            entity_chunk_type = entity_chunk[1]
            entity_chunk = (entity_chunk_text+' '+token.text,entity_chunk_type)


    uninformative_entities = ['DATE','TIME','QUANTITY','ORDINAL','CARDINAL','QUANTITY','MONEY','PERCENT','PERSON']

    named_entities = list(filter(lambda x: True if x[1] not in uninformative_entities else False, named_entities))
    named_entities = np.unique(named_entities,axis=0) 

    return named_entities

In [159]:
doc_df = pd.read_csv('tables/doc_69_76.csv')
id_to_text_list = doc_df['id_to_text'].values
free_text_list = doc_df['text'].values

In [ ]:
#plan
# named entities
# lemmatize -> only include named entities occuring more than k

# keywords
# using named entities as stop words for keyword extraction? -> try this
# lemmatization before keyword extraction!
# similarly: only include keywords occuring more than x
# try for sure keybert

# extension
# doing the entire process with only ne, and then maybe with only keywords with specific pos tags???

In [308]:
if os.path.isfile('ne2vec/ner_dict'):
    with open("ne2vec/ner_dict", "rb") as fp:
        ner_dict = pickle.load(fp)
    print('ner dict loaded.')

else:
    ner_dict = {}

    for idx,text in enumerate(free_text_list):

        if not(isinstance(text, float) and math.isnan(text)): # check if NaN
            id_to_text = id_to_text_list[idx]
            ne_list = apply_ner(text)

            for ne_tuple in ne_list:
                ne = ne_tuple[0]

                if not ner_dict.get(ne,None):
                    ner_dict[ne] = [id_to_text]
                else:
                    ner_dict[ne].append(id_to_text)
    
    with open('ne2vec/ner_dict', 'wb') as f:
        pickle.dump(ner_dict, f)
    print('ner dict computed and saved.')


min_ne_count = 50
copy_ner_dict = copy.deepcopy(ner_dict)

for key in copy_ner_dict:
    
    if len(ner_dict[key]) < min_ne_count:
        del ner_dict[key]

ner dict loaded.


In [310]:
ne2doc_df = pd.DataFrame(columns=['id_to_text','named_entity'])


for key in ner_dict:

    id_list = ner_dict[key]

    for id in id_list:

        ne2doc_df = pd.concat((ne2doc_df, 
                                pd.DataFrame({'id_to_text':[id],'named_entity':[key]})),
                                ignore_index=True)

ne2doc_df.to_parquet('ne2vec/ne2doc_df.parquet')
# DONE.

KeyboardInterrupt: 

In [ ]:
#EXPERIMENT
#lemmatization procedure (maybe used)

lemmatized_ner_dict = {}

for key in ner_dict:

    lemmatized_ne = " ".join([token.lemma_ for token in nlp(key.lower())])
    lemmatized_ner_dict[lemmatized_ne] = lemmatized_ner_dict.get(lemmatized_ne,0)+1

In [223]:
# exclude found named entities to avoid duplication
stop_words=text.ENGLISH_STOP_WORDS.union(list(map(lambda x: x.lower(),list(ner_dict.keys()))))

custom_kw_extractor = yake.KeywordExtractor(lan='en', n=1, top=20,dedupLim=0.9, stopwords=stop_words)

custom_voc = set()
keyword_docs = []

for free_text in tqdm(free_text_list):
    if not(isinstance(free_text, float) and math.isnan(free_text)): # check if NaN
        keywords = custom_kw_extractor.extract_keywords(nlp(free_text.lower()).text)
        keywords = set(map(lambda x: x[0],keywords))

        keyword_docs.append(" ".join(keywords))
        custom_voc.update(keywords)

100%|██████████| 19512/19512 [1:04:18<00:00,  5.06it/s]


In [224]:
with open('ne2vec/keyword_docs', 'wb') as f:
        pickle.dump(keyword_docs, f)
with open('ne2vec/custom_voc', 'wb') as f:
        pickle.dump(custom_voc, f)

In [275]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(max_df=0.2,min_df=30)

X = vectorizer.fit_transform(keyword_docs)

processed_key2doc_mat = vectorizer.inverse_transform(X)

In [297]:
ke2doc_df = pd.DataFrame(columns=['id_to_text','named_entity']) # 'named_entity' is actually keyword/


for idx,temp_keyword_list in enumerate(tqdm(processed_key2doc_mat)):

    for temp_keyword in temp_keyword_list:

        ke2doc_df = pd.concat((ke2doc_df, 
                                pd.DataFrame({'id_to_text':[idx],'named_entity':[temp_keyword]})),
                                ignore_index=True)

ke2doc_df['id_to_text'] = ke2doc_df['id_to_text'].astype(str)
ke2doc_df.to_parquet('ne2vec/ke2doc_df.parquet')
# DONE.

100%|██████████| 19511/19511 [22:49<00:00, 14.25it/s]


In [322]:
gds = GraphDataScience("bolt://localhost:7687", auth=('neo4j', 'bos'), database='keyword2vec')

embedding_df = gds.run_cypher(
    """
        match (e:Entity)
        return e.name as entity, e['fastrp-embedding'] as fastrp_embedding, e['node2vec-embedding'] as node2vec_embedding
    """
)

In [336]:
from sklearn.manifold import TSNE
reduced_emb_mat = TSNE(n_components=2, perplexity=50).fit_transform(np.stack(embedding_df['node2vec_embedding']))
x,y = reduced_emb_mat[:,0],reduced_emb_mat[:,1]

'''matplotlib
fig, ax = plt.subplots(figsize=(20, 20))
ax.scatter(x,y)
for i, txt in enumerate(embedding_df['entity'].values):
    ax.annotate(txt, (x[i], y[i]))'''

# plotly
fig = px.scatter(x=x, y=y, text=embedding_df['entity'].values, width=900, height=900)
fig.write_html("ne2vec/only_ke_emb_node2vec.html")
#fig.show()

/Users/gokberk/miniconda3/envs/ml4hc_project2/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:780: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

/Users/gokberk/miniconda3/envs/ml4hc_project2/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:790: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.



In [347]:
cossim_mat = cosine_similarity(np.stack(embedding_df['fastrp_embedding']))

In [348]:
def most_similar(word, top_n):

    word_idx = embedding_df[embedding_df['entity']==word].index[0]

    similar_entity_idx = np.argsort(cossim_mat[word_idx])[::-1][1:top_n+1]

    similar_entity_names = embedding_df['entity'].values[similar_entity_idx]
    similar_entity_sims = cossim_mat[word_idx][similar_entity_idx]

    return np.array([similar_entity_names,similar_entity_sims]).T

In [352]:
most_similar('change',10)

array([['given', 0.9997970545229851],
       ['private', 0.9997787087634654],
       ['end', 0.9997768219522268],
       ['progress', 0.9997757420410192],
       ['strong', 0.999774906026226],
       ['terms', 0.9997710754629248],
       ['case', 0.9997648636837971],
       ['expressed', 0.9997574591859688],
       ['fact', 0.9997565880692844],
       ['leaders', 0.9997526278924671]], dtype=object)